In [69]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
import os
import shutil
from pathlib import Path
import os.path
import subprocess
import os
import glob
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

In [ ]:
def getCatData():
    # display nightowls categories and supercategories
    cats = COCO.loadCats(coco.getCatIds())
    cat_map = {}
    for i,cat in enumerate(cats):
        cat_map[cat['id']] = i
    nms=[cat['name'] for cat in cats]
    catIds = coco.getCatIds()
    return catIds, cat_map

In [ ]:
def get_imgs_from_each_category(catIds, numImgs=None):
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:numImgs]
        if(len(imgIds_current_batch) < 30):
            print('ERROR! Not 30 images in this category!')
        imgIds = imgIds + imgIds_current_batch
    return imgIds

In [71]:
"""
getImgIds() -> list[imgId] 
loadImgs(ids : [imgId]) -> list[img]
getAnnIds(imgIds: list[imgId]) -> list[annId]
loadAnns(ids: list[annId]) -> list[ann]

*One imgId can be multiple annIds
*One annId is one ann


#imgIds need to have annotations and those annotations bboxes must not be negative
#write_label_files need
    #filename, which are in imgs
    #catIds, which are in anns
    #bboxes, which are in anns
"""
def get_random_img_ids(numImgs):
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    #I think NO is the one I needed to filter for
    imgIds = list(filter(imgId_has_annotation, imgIds))
    imgIds = list(filter(imgId_has_positive_bboxes, imgIds))
    return imgIds[:numImgs]

def get_img_ids_per_cat(numImgs_per_cat):
    catIds = coco.getCatIds()
    imgIds_total = []
    for catId in catIds:
        imgIds_cat = coco.getImgIds(catIds=[catId])
        #imgIds = coco.getImgIds()
        random.shuffle(imgIds_cat)
        imgIds_current = []
        for imgId in imgIds_cat:
            if imgId_has_annotation(imgId) and imgId_has_positive_bboxes(imgId):
                imgIds_current.append(imgId)
            if(len(imgIds_current) == numImgs_per_cat):
                break
        imgIds_total = imgIds_total + imgIds_current
    return imgIds_total

def imgId_has_positive_bboxes(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        negative_bbox = False 
        for bbox_val in ann['bbox']:
            if bbox_val < 0:
                negative_bbox = True
        if negative_bbox == True:
            return False
    return True

def imgId_has_annotation(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    if(len(annIds) > 0):
        return True
    else:
        return False

#def write_label_files(numImgs_per_cat):
def write_label_files(numImgs, cat_map, catIds, label_dir1, coco):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    #imgIds = get_img_ids_per_cat(numImgs_per_cat)
    imgIds = get_random_img_ids(numImgs)
    print(len(imgIds) == 98)

    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path(label_dir1)
                        #'/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels'
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            #assert len(annIds) != 0
            anns = coco.loadAnns(annIds)
            #if not anns:
                #print('no anns for this file')
            #print(len(anns))
            #print('-----------------------------')
            #anns = remove_negative_bbox_anns(anns)
            for i,ann in enumerate(anns):
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                #print(label_dir/filename)
                #print(x_center, y_center, width, height)
                #print('\n')
                file.write(f'{cat_map[ann["category_id"]]} {x_center} {y_center} {width} {height}\n')
    return

def copy_images_to_dataset_dir(src_image_dir, dest_image_dir, image_extension, src_label_dir):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir(src_label_dir)
    for f in fs:
        if f == '.DS_Store':
            continue
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
        else:
            print(f'{src} is not a file')
    return

def remove_files(img_dir, label_dir):
    files = glob.glob(f'{img_dir}*')
    for f in files:
        os.remove(f)
    files = glob.glob(f'{label_dir}*')
    for f in files:
        os.remove(f)

#Currently outputting 473 instead of 480 (4*120)
#So one of more categories has fewer than 120 valid images, it *seems*
if __name__ == "__main__":
    #os.system('rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*')


    #region Nightowls Training -----------------------------------------------------
    def produce_nightowls_training_set():
        """
        This was done on purple!!
        """
        exit()
        label_dir =  '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels/'
        src_img_dir = '/Users/azakaria/Documents/yolo datasets/nightowls/val2017'
        img_dir = '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'
        annotations_json_file = '/Users/azakaria/Documents/yolo datasets/nightowls/instances_val2017.json'

        coco=COCO('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/nightowls_validation')#


        coco=COCO('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_training.json')
        catIds, cat_map = getCatData()
        write_label_files(480, cat_map, catIds, '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/dataset/labels') 
        copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'), '.png', '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/dataset/labels') #dataset/images
    #endregion


    #region Nightowls validation -----------------------------------------------------
    def produce_nightowls_validation_set(numImgs):

        label_dir =  '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels/'
        src_img_dir = '/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'
        img_dir = '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'
        annotations_json_file = '/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation.json'

        remove_files(img_dir,label_dir)

        coco=COCO(annotations_json_file)
        print(annotations_json_file)
        catIds, cat_map = getCatData()
        write_label_files(numImgs, cat_map, catIds, label_dir, coco) 
        copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.jpg', label_dir) #dataset/images

        os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')
    #endregion

    produce_nightowls_validation_set(480)

    def produce_coco_training_set(numImgs):
        label_dir =  '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_val1/dataset/labels/'
        src_img_dir = '/Users/azakaria/Documents/yolo datasets/coco/val2017'
        img_dir = '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_val1/dataset/images/'
        annotations_json_file = '/Users/azakaria/Documents/yolo datasets/coco/instances_val2017.json'

        remove_files(img_dir,label_dir)

        coco=COCO(annotations_json_file)
        catIds, cat_map = getCatData()
        write_label_files(numImgs, cat_map, catIds, label_dir) 
        copy_images_to_dataset_dir(Path(src_img_dir),Path(img_dir), '.jpg', label_dir) #dataset/images

        os.system(f'ls {img_dir} | wc -l ; ls {label_dir} | wc -l')
    #produce_coco_training_set(480)


    #region COCO validation
    #coco=COCO('/Users/azakaria/Documents/yolo datasets/coco/instances_val2017.json')
    #catIds, cat_map = getCatData()
    #write_label_files(98, cat_map, catIds, '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_val1/dataset/labels') 
    #copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/coco/val2017'),Path('/Users/azakaria/Code/openmpf-yolo-training/coco/coco_val1/dataset/images/'), '.jpg', '/Users/azakaria/Code/openmpf-yolo-training/coco/coco_val1/dataset/labels') 
    #endregion

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation.json
False
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000013546.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000195045.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000574702.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000300233.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000478474.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000514586.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000365642.jpg is not a file
/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation/000000107094.jpg is not a file
/Users/azakaria/Doc

In [ ]:
print(catIds, cat_map)
imgIds = get_random_img_ids(98)
t = []
for imgId in imgIds:
    #annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
    annIds = coco.getAnnIds(imgIds = imgId)
    if not annIds:
        print(f'No annotation: {imgId}')
        t.append(annIds)
        #print('not annIds')
    else:
        print(f'Yes annotation: {imgId}')
print(len(t))